In [31]:
import os
from pathlib import Path
from typing import Sequence
from pylovepdf.ilovepdf import ILovePdf
from pylovepdf.tools.merge import Merge


In [29]:
_my_public_key = "<INSERT YOUR PUBLIC KEY HERE>"

## function to compress PDF

In [30]:

def compress_pdf(input_pdf: Path, compression_lvl: str = "low") -> None:
    """
    this function compresses the
    input PDF and writes them
    to given output path.

    Args:
    ----
        input_pdf: (Path) - input PDF path with file.
        compression_lvl: (str) - compression level
        to reduce pdf file size. Defaults to low.
        acceptable values are low, extreme and
        recommended.
    """
    # check input file existence:
    if not os.path.isfile(input_pdf):
        raise FileNotFoundError(f"PDF {input_pdf} doesn't exist. please check path!")

    # get input file name:
    file_name = os.path.splitext(os.path.basename(input_pdf))[0]
    folder_name = os.path.dirname(input_pdf)
    
    # initialize ilovepdf:
    ilovepdf = ILovePdf(_my_public_key, verify_ssl=True)

    # create compress task:
    task = ilovepdf.new_task("compress")
    task.add_file(input_pdf)
    task.debug = False
    task.compression_level = compression_lvl
    task.set_output_folder(folder_name)

    # start task:
    task.execute()
    task.download()
    task.delete_current_task()


## function to merge pdf

In [34]:
def merge_pdf(pdf_files: Sequence[Path], merged_pdf_path: Path = './merged_output/') -> None:
    """
    this function can be used to merge pdf
    files provided in the same sequence as 
    passed to argument pdf_files

    Args:
        pdf_files (Sequence[Path]): pdf files to be
        merged.
    """
    if len(pdf_files) < 2:
        raise ValueError("Minimum two pdf files required for merge")

    # initialize merge task:
    task = Merge(_my_public_key, verify_ssl=True, proxies=None)
    
    # merge files provided in pdf_files in same seq:
    for _file in pdf_files:
        task.add_file(_file)

    task.debug = False
    task.set_output_folder(merged_pdf_path)
    
    # start task:
    task.execute()
    task.download()
    task.delete_current_task()

#### Passing input pdf file from ../pdf-doc folder to create compressed pdf output with reduced size:

In [28]:
compress_pdf(input_pdf='/Users/sukakrish/krish-workspace/python-pdfs/pdf-doc/docs-pdf/python-basics-sample-chapters.pdf', compression_lvl="recommended")

Uploading file...
File uploaded! Below file stats:
download_filename: python-basics-sample-chapters_compress_02-01-2023.pdf 
filesize: 1556490 
output_extensions: ["pdf"] 
output_filenumber: 1 
output_filesize: 770870 
status: TaskSuccess 
timer: 0.906 

File downloaded!
Task delete status: 200 



#### Merge two or more pdf files

In [36]:
pdf_files = ['./pdf-doc/docs-pdf/test_1.pdf', './pdf-doc/docs-pdf/test_2.pdf']

merge_pdf(pdf_files=pdf_files)

Uploading file...
File uploaded! Below file stats:
download_filename: merged_merge_02-01-2023.pdf 
filesize: 82731 
output_extensions: ["pdf"] 
output_filenumber: 1 
output_filesize: 0 
status: TaskSuccess 
timer: 0.148 

File downloaded!
Task delete status: 200 

